In [58]:
import tweepy
import re
import pickle
import pandas as pd
import statistics
import plotly
import plotly.graph_objects as go
import operator
import matplotlib.pyplot as plt

In [59]:
consumer_key = 'cyXcjxUc1ZlLD2v1DjaJrLiVI' 
consumer_secret = 'pD3mk041ROBiSSwt2C3kytR9eF1zYrX1vBD8whpDln5Zq4y1jX' 
access_key = '1279253779540410373-8ODCqefNcIvSFXSrwKH7DPhtQTda3i' 
access_secret = 'nVGU8WyEdGCQ2gU1qQ57NXxLpYQts4Emt1UPM6GVlYaav'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
tweets = api.user_timeline(screen_name="dtptraffic", 
                           # 200 is the maximum allowed count
                           count=200,
                           #include_rts = False,
                           # Necessary to keep full_text 
                           # otherwise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
all_tweets = []
all_tweets.extend(tweets)
oldest_id = tweets[-1].id

In [ ]:
while len(all_tweets)<=10000:
    tweets = api.user_timeline(screen_name="dtptraffic", 
                           # 200 is the maximum allowed count
                           count=200,
                           #include_rts = False,
                           max_id = oldest_id - 1,
                           # Necessary to keep full_text 
                           # otherw hiise only the first 140 words are extracted
                           tweet_mode = 'extended'
                           )
    if len(tweets) == 0:
        break
    oldest_id = tweets[-1].id
    all_tweets.extend(tweets)
    print('N of tweets downloaded till now {}'.format(len(all_tweets)))

N of tweets downloaded till now 400
N of tweets downloaded till now 600
N of tweets downloaded till now 800
N of tweets downloaded till now 1000
N of tweets downloaded till now 1200
N of tweets downloaded till now 1400
N of tweets downloaded till now 1600
N of tweets downloaded till now 1800
N of tweets downloaded till now 2000
N of tweets downloaded till now 2200
N of tweets downloaded till now 2400
N of tweets downloaded till now 2600
N of tweets downloaded till now 2800
N of tweets downloaded till now 3000
N of tweets downloaded till now 3200
N of tweets downloaded till now 3250


In [56]:
infile = open("/content/drive/MyDrive/psosm/section1",'rb')
all_tweets = pickle.load(infile)
infile.close() 

In [57]:
f='sec1.txt'
cnt=0
with open(f,"a") as file:
      for row in all_tweets:
        file.write(str(row.id))
        file.write("\n")
      file.close()

In [104]:
df=pd.DataFrame([[tweet.id,tweet.full_text,tweet.in_reply_to_status_id_str,tweet.created_at] for tweet in all_tweets])
df.columns=['id','text','in_reply_id','time_created']
#df.head()

In [87]:
def get_pii(reply_text):
  numbers={}
  adhar={}
  email={}
  pin={}
  vn={}
  cc={}
  for i in reply_text:
    #t=re.findall("^(?:(?:\+|0{0,2})91(\s*[\-]\s*)?|[0]?)?[789]\d{9}$",i)
    t=re.findall(r'\b[6-9]\d{9}\b',i)
    if len(t)!=0:
      numbers[i]=t
    t=re.findall(r'\b\d{4}\s\d{4}\s\d{4}\b',i)
    if len(t)!=0:
      adhar[i]=t
    t=re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+",i)
    if len(t)!=0:
      email[i]=t
    t=re.findall(r'\b[1-9]{1}[0-9]{5}\b',i)
    if len(t)!=0:
      pin[i]=t
    t=re.findall(r'\b[A-Z]{2}\d{1,2}[A-Z]{2,3}\d{4}\b',i)
    if len(t)!=0:
      vn[i]=t
    t=re.findall(r'((?:(?:\\d{4}[- ]?){3}\\d{4}|\\d{15,16}))(?![\\d])',i)
    if len(t)!=0:
      cc[i]=t

  print('Phone numbers:',len(numbers))
  print('Adhar numbers: ',len(adhar))
  print('Emails: ',len(email))
  print('Pin Codes: ',len(pin))
  print('Vehicle Numbers: ',len(vn))
  print('credit cards: ',len(cc))

In [90]:
reply_text=[]
reply_textId=[]
reply_t={}
for tw in all_tweets:
      reply_t[tw.in_reply_to_status_id_str]=tw.created_at
reply_id=list(df[df['in_reply_id']!=None]['in_reply_id'])
original_tt={}
for i in range(0,3250,50):
    t= api.statuses_lookup(reply_id[i:i+50],tweet_mode = 'extended')
    for tw in t:
      original_tt[tw.id_str]=tw.created_at      
      reply_text.append(tw.full_text)    
      reply_textId.append([tw.id_str,tw.full_text])
get_pii(reply_text)

Phone numbers: 11
Adhar numbers:  0
Emails:  3
Pin Codes:  56
Vehicle Numbers:  141
credit cards:  0


In [91]:
csv_df=pd.DataFrame(reply_textId)
csv_df.to_csv('sample_30.csv')

In [99]:
df = pd.read_csv ('sample_30.csv')
df.columns=['id','text','label']
df

,id,text,label
0,1.383030e+18,@dtptraffic @dtptraffic is Tikri Border open f...,assistance
1,1.382990e+18,"@dtptraffic @CPDelhi Dear Sir, intrastate tra...",assistance
2,1.383030e+18,@dtptraffic sir they are not using mask and no...,report
3,1.382940e+18,@DelhiPolice are we allowed to travel from NCR...,assistance
4,1.382970e+18,@dtptraffic @DCPNEastDelhi Sir I need to take...,assistance
5,1.382940e+18,@Virender9250 @dtptraffic @ArvindKejriwal @LtG...,report
6,1.382960e+18,"@DelhiPolice @dtptraffic Dear Sir, We are an ...",assistance
7,1.383020e+18,@dtptraffic @ghaziabadpolice @ArvindKejriwal C...,request
8,1.382950e+18,"@dtptraffic @CPDelhi Sir, what I have to do if...",assistance
9,1.383010e+18,@dtptraffic one police man drive without helme...,report


In [75]:
reply_text=[]
reply_id=list(df[df['in_reply_id']!=None]['in_reply_id'])
cnt=0
for i in range(0,3250,50):
    t= api.statuses_lookup(reply_id[i:i+50],tweet_mode = 'extended')
    for tw in t:
      if 'media' in tw.entities:          
        reply_text.append(tw.full_text)
        cnt=cnt+1
print(cnt)
print(get_pii(reply_text))

1639
Phone numbers: 2
Adhar numbers:  0
Emails:  0
Pin Codes:  33
Vehicle Numbers:  125
credit cards:  0
None


**question 2**

> Indented block



In [9]:
import datetime
time_ser={}
times=[]
for i in list(reply_t.keys()):
  if i in original_tt.keys():
    time_ser[original_tt[i].strftime("%Y-%m-%d-%H-%M")]=(reply_t[i]-original_tt[i]).seconds
    times.append((reply_t[i]-original_tt[i]).seconds)
    #print(reply_t[i]-original_tt[i])
print(len(times))


3024 2894
2894


In [77]:
print(min(times))
print(max(times))
print(statistics.mean(times))
print(statistics.pstdev(times))
print(len(time_ser))
fig =[go.Scatter(x=list(time_ser.keys()), y=list(time_ser.values()),)]
plotly.offline.iplot(fig)

1
86190
1876.5694540428474
6436.3208862389465
2742
